In [1]:
import numpy as np 

X_train_c, X_test, Y_train_c, Y_test = np.load('./npy/classify_train.npy', allow_pickle=True)

print('X_train', X_train_c.shape)
print('X_test ', X_test.shape)
print('Y_train', Y_train_c.shape)
print('Y_test ', Y_test.shape)

X_train (10864, 128, 128, 1)
X_test  (1208, 128, 128, 1)
Y_train (10864, 1)
Y_test  (1208, 1)


In [2]:
import keras

c_model = keras.models.load_model('./models/class_c_model_p5_40.h5')
d_model = keras.models.load_model('./models/gan_d_model.h5')
g_model = keras.models.load_model('./models/whole_model.h5')

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.




/home/hiwony/.local/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


ValueError: axes don't match array

In [ ]:
# Util 함수

def show_result(target_data=X_test, count=10, threshold=0.2):

    shuffled_index = np.random.permutation(X_test.shape[0])
    shuffled_index = np.arange(0,count)
    model_input_shape = (1,128, 128, 1)

    for i in range(count):
        cl_ = Y_test[i]
        target_input = target_data[shuffled_index[i]]
        c_out = c_model.predict(target_input.reshape(model_input_shape)) 
        g_out = g_model.predict(target_input.reshape(model_input_shape))
        d_out = d_model.predict(g_out)
        c_out_g_in = c_model.predict(g_out)
        
        diff = np.absolute(g_out[0] - target_input)
        diff[diff < threshold] = 0.0

        # 실제 이미지
        if Y_test[i] == 0 :
            print('Input is classified as Normal Image %.1f' % Y_test[i], '(0 : Normal, 1 : Fracture)')
        else : 
            print('Input is classified as Fracture Image %.1f' % Y_test[i], '(0 : Normal, 1 : Fracture)')

        # classify  0 normal, 1 fracture 
        if c_out[0][1] <= 0.5 :  
            print('Classified as Normal Image %.2f' % c_out[0][1], '(0 : Normal, 1 : Fracture)')
        else :
            print('Classified as Fractrue Image %.2f' % c_out[0][1], '(0 : Normal, 1 : Fracture)')
        
        # discriminator 0 real 1 fake 디스크리미네이터의 진짜와 가짜
        d_out_ = np.argmax(d_out, axis = 1)
        if d_out_ <= 0.5 :  
            print('Generated is Discriminated as Real Image %.2f' % d_out_, '(0 : Real, 1 : Fake)')
        else : 
            print('Generated is Discriminated as Fake Image %.2f' % d_out_, '(0 : Real, 1 : Fake)')            
        
        # generator_classify 0 normal, 1 fracture
        if c_out_g_in[0][1] <= 0.5 :  
            print('Generated is classified as Normal Image %.2f' % c_out_g_in[0][1], '(0 : Normal, 1 : Fracture)')
        else :
            print('Generated is classified as Fracture Image %.2f' % c_out_g_in[0][1], '(0 : Normal, 1 : Fracture)')
        
            
        plt.figure(figsize=(10,10))
        plt.subplot(1,3,1)
        plt.title('Input img')
        plt.axis('off')
        plt.imshow(target_input.reshape((128,128)),cmap='gray' , vmin=0, vmax=1)
        
        plt.subplot(1,3,2)
        plt.title('Generated img')
        plt.axis('off')
        plt.imshow(g_out[0].reshape((128,128)),cmap='gray', vmin=0, vmax=1)
        
        plt.subplot(1,3,3)
        plt.title('Fracture area')
        plt.axis('off')
        plt.imshow(diff.reshape((128,128)),cmap='gray')
        plt.show()
        plt.close()
        print()
#cmap=plt.cm.binary
# show_result(X_test)

In [ ]:
show_result(X_test,threshold = 0.2)